In [2]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np
import json


import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as wd2

In [3]:
colorway = ['#702632', '#A4B494', '#495867', '#912F40', "#81909E", "#F4442E", "#DB7C26", "#BB9BB0"]
pio.templates["cardinal"] = go.layout.Template(
    layout=go.Layout(
        colorway=colorway
    ))
# combine templates 
pio.templates.default = 'plotly_white+cardinal'

## real data 

In [4]:
a00, a01 = h.import_desired_data("A", "15T")
# print(a00["Window Open"].unique(), a01["Window Open"].unique(), a00.columns)

In [10]:
w1 = wd2.Window_Detect2(a01)
w1.analyze_window_change(wd2.make_stl_smooth)

w2 = wd2.Window_Detect2(a01)
w2.analyze_window_change(wd2.make_sin_smooth)

w3 = wd2.Window_Detect2(a01)
w3.analyze_window_change(wd2.make_ewm_smooth)

In [33]:
def plot_many_dist(objects, title_arr):
    # triple plot for distributions 
    fig = make_subplots(rows=1, cols=len(objects), shared_yaxes=True, subplot_titles=title_arr)
    marker_width = 0.1 
    bin_size = 0.003

    for obj_ix, obj in enumerate(objects):
        leg = True if obj_ix == 0 else False
        for ix, ser in enumerate([obj.deriv2, obj.deriv]):
            opacity = 0.9 if ix == 0 else 1
            color = '#702632' if ix == 0 else '#A4B494'
            fig.add_trace(go.Histogram(
            x=ser, histnorm='probability', name=f' Deriv{2 - ix}', opacity=opacity, marker_line=dict(width=marker_width ,color='black'), xbins=dict(size=bin_size), marker_color=color, showlegend=leg), row = 1, col = obj_ix+1)

    fig.update_layout(barmode="stack")

    return fig


In [34]:
fig = plot_many_dist([w1, w2, w3], ["STL", "Sinusoid", "EWM"])
fig.show()

In [6]:
# triple plot for distributions 
fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=["STL", "Sinusoid", "EWM"])
marker_width = 0.1 
bin_size = 0.003

for obj_ix, obj in enumerate([w1, w2, w3]):
    leg = True if obj_ix == 0 else False
    for ix, ser in enumerate([obj.deriv2, obj.deriv]):
        opacity = 0.9 if ix == 0 else 1
        color = '#702632' if ix == 0 else '#A4B494'
        fig.add_trace(go.Histogram(
        x=ser, histnorm='probability', name=f' Deriv{2 - ix}', opacity=opacity, marker_line=dict(width=marker_width ,color='black'), xbins=dict(size=bin_size), marker_color=color, showlegend=leg), row = 1, col = obj_ix+1)

fig.update_layout(barmode="stack")


In [18]:
def make_dual_plot_abstract(time, objects, names, traces, title_arr, mode="lines"): 
    fig = make_subplots(rows=1, cols=len(objects), shared_yaxes=True, subplot_titles=tuple(title_arr))
    
    series = [[getattr(obj, trace) for trace in traces] for obj in objects]


    opacities = [0.4] + [1]*(len(series[0])-1)

    for series_ix, s in enumerate(series):
        leg = True if series_ix == 0 else False
        for ix, name, ser in zip(range(len(names)), names, s):
            if mode != "lines": 
                # caveat for plotting guesses 
                time_guess = time if ix+1 != len(s) else objects[series_ix].guess_times

                fig.add_trace(go.Scatter(x=time_guess, y=ser, name=name, mode=mode[ix], line_color=colorway[ix], opacity=opacities[ix],legendgroup=name, showlegend=leg), row = 1, col = series_ix+1)
            else:
                fig.add_trace(go.Scatter(x=time, y=ser, name=name, mode=mode, line_color=colorway[ix], opacity=opacities[ix],legendgroup=name, showlegend=leg), row = 1, col = series_ix+1)

    return fig, names






In [26]:
all_names = ["Window", "Observed Temp", "Smoothed", "Difference", "Deriv1", "Deriv2"]
all_traces = ["window_norm", "temp_norm", "smooth_series", "dif", "deriv", "deriv2"]

fig, names = make_dual_plot_abstract(time=w1.time, objects=[s1, w1], names=all_names, traces=all_traces, title_arr=["Simulation", "Measurement"], )
fig.show()

In [35]:
fig, names = make_dual_plot_abstract(time=w1.time, objects=[s1], names=all_names, traces=all_traces, title_arr=["Simulation",], )
fig.show()

In [27]:
fig = wd2.update_dual_plot(fig,  names, ["Window", "Deriv1",],)
fig.show()

In [19]:
fig, names = make_dual_plot_abstract(time=w1.time, objects=[w1, w2, w3], names=["Window", "Z-Score 1",], traces=["window_norm", "zscore",], title_arr=["STL", "Sinusoid", "EWM"], mode="lines")
fig.show()

In [20]:
fig, names = make_dual_plot_abstract(time=w1.time, objects=[w1, w2, w3], names=["Window", "Z-Score", "Guesses"], traces=["window_norm", "zscore_norm", "guess_values"], title_arr=["STL", "Sinusoid", "EWM"], mode=["lines", "lines", "markers"])
fig.show()

In [22]:
fig, names = make_dual_plot_abstract(time=w1.time, objects=[w1, w2, w3], names=["Window", "Z-Score", "Guesses"], traces=["window_norm", "zscore_norm", "guess_values"], title_arr=["", "", ""], mode=["lines", "lines", "markers"])
fig.show()

In [37]:
# objects=[w1, w2, w3]
# names=["Window", "Z-Score", "Guesses"]
# traces=["window_norm", "zscore", "guess_values"]

# series = [[getattr(w1, "time") for trace in traces] for obj in objects]

In [13]:
fig, names = make_dual_plot_abstract(time=w1.time, objects=[w1, w2, w3], names=["Window", "Guesses"], traces=["window_norm", "guess_values"], title_arr=["STL", "Sinusoid", "EWM"], mode=["lines", "markers"])
fig.show()

In [9]:
fig, names = make_dual_plot_abstract(time=w1.time, objects=[w1, w2, w3], names=["Window", "Z-Score 1", "Z-Score 2"], trace_type="zscore", title_arr=["STL", "Sinusoid", "EWM"])
fig.show()

In [31]:
def make_dual_plot(obj1, obj2): 
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True)

    s2, s3 = [[obj.window_norm, obj.temp_norm, obj.smooth_series, obj.dif, obj.deriv, obj.deriv2] for obj in [obj1, obj2]]

    names = ["Window", "Observed Temp", "Smoothed", "Difference", "Deriv1", "Deriv2"]
    
    opacities = [0.4] + [1]*(len(s2)-1)

    for ix, name, ser in zip(range(len(names)), names, s2):
        fig.add_trace(go.Scatter(x=w2.time, y=ser, name=name, mode='lines', line_color=h.colorway[ix], opacity=opacities[ix],legendgroup=name,), row = 1, col = 1)

    for ix, name, ser in zip(range(len(names)), names, s3):
        fig.add_trace(go.Scatter(x=w2.time, y=ser, name=name, mode='lines', line_color=h.colorway[ix], opacity=opacities[ix],legendgroup=name, showlegend=False), row = 1, col = 2)

    return fig, names

In [32]:
def update_dual_plot(fig, names, show_arr, ):
    for name in names: 
        if name in show_arr:
            fig.update_traces(visible=True, selector=dict(name=name))
        else:
            fig.update_traces(visible="legendonly", selector=dict(name=name))
    return fig
   


In [73]:
fig, names = make_dual_plot(w2, w3)
update_dual_plot(fig,  names, ["Window", "Observed Temp", "Smoothed"],)

In [75]:
update_dual_plot(fig, names, ["Deriv1", "Deriv2"])

## simulation data 

In [25]:
# sim_data_path = "../../data/energy_model/230307.pkl"
# sim_data = pd.read_pickle(sim_data_path)

# # update year to match ! NOTE: this has not been calibrated, so based on EPW data from 2017!
# delta = pd.Timestamp('2022-01-01') - pd.Timestamp('2017-01-01')
# sim_data["DateTime"] += delta 

# # trim so match real data 
# start = a01["DateTime"][0]
# end = a01["DateTime"].iloc[-1]
# mask = (sim_data['DateTime'] >= start) & (sim_data['DateTime'] <= end)
# sim_data = sim_data.loc[mask]

# sim_data.to_pickle("../../data/energy_model/230307_match.pkl")

In [23]:
sim_data_path = "../../data/energy_model/230307_match.pkl"
sim_data = pd.read_pickle(sim_data_path)

# smoothing on different time series than the observation 
sim_smooth = wd2.make_stl_smooth(series=sim_data["Always Closed"], time=None, model="additive")

# rename label in simulation dataframe to match class
sim_data.rename(columns={"Varied": "Temp C"}, inplace=True)

In [24]:
s1 = wd2.Window_Detect2(sim_data)
s1.analyze_window_change(smooth_fx=None, sim_smooth=sim_smooth)
